In [ ]:
import numpy as np
import pandas as pd
from datasets import load_dataset

In [ ]:
SPLIT = "train"

ds = load_dataset("aholovko/go-critic-style")
print(ds)

In [ ]:
class_label = ds.features["labels"].feature
label_names = class_label.names

In [ ]:
labels_list = ds["labels"]

In [ ]:
# Flatten and count labels
all_labels = [lbl for sublist in labels_list for lbl in sublist]
label_counts = pd.Series(all_labels).value_counts().sort_index()
print(f"\n=== {SPLIT.upper()} SPLIT ===")
print("Label counts:")
for label_id, count in label_counts.items():
    name = label_names[label_id]
    print(f"  {label_id} ({name}): {count}")

In [ ]:
# Cardinality: number of labels per example
cardinality = pd.Series([len(lst) for lst in labels_list])
print("\nLabel cardinality distribution (labels per example):")
print(cardinality.value_counts().sort_index().to_string())
print("\nCardinality stats:")
print(cardinality.describe().to_string())

In [ ]:
# Co-occurrence matrix
num_labels = len(label_names)
co_matrix = np.zeros((num_labels, num_labels), dtype=int)
for lst in labels_list:
    for i in lst:
        for j in lst:
            co_matrix[i, j] += 1

co_df = pd.DataFrame(co_matrix, index=label_names, columns=label_names)
print("\nLabel co-occurrence matrix:")
print(co_df.to_string())
print("\n" + "#" * 50)